In [262]:
from collections import deque
import logging

from numpy import exp, inf, log
from numpy.random import rand
import numpy as np

## Logging Config

In [263]:
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Random Variable Helpers

In [264]:
def gen_exp_rv(ld):
    return -1 / ld * log(1 - rand()) # using inverse CDF of exp RVs

def gen_exp_rv_hourly_rate(hours):
    return gen_exp_rv(hours) * 60 * 60

## Classes

### Stall
Handles queue and departure of customers

In [265]:
class Stall:
    def __init__(self, simulator, mu):
        self.mu = mu
        self.simulator = simulator
        self.queue = deque([])
        self._departure_times = deque([])
        
    def add_customer(self, customer):
        self.queue.append(customer)
        
        if not self._departure_times:
            next_departure_time = gen_exp_rv(self.mu) + self.simulator.time
        else:
            next_departure_time = gen_exp_rv(self.mu) + self._departure_times[-1]
            
        self._departure_times.append(next_departure_time)
        
    def serve_customer(self):
        if self.queue:
            self.queue.popleft()
            self._departure_times.popleft()
        else:
            raise IndexError('Queue is empty')
            
    @property
    def next_departure_time(self):
        if not self._departure_times:
            return inf
        return self._departure_times[0]

### Customer and Customer Generator
Handles arrival of customers

In [266]:
class Customer:
    def __init__(self, arrival_time):
        self.arrival_time = arrival_time

# handles arrival of customers 
class CustomerGenerator:
    def __init__(self, simulator, ld):
        self.simulator = simulator
        self.ld = ld
        self._next_arrival_time = -1
        
    def generate_customer(self):
        return Customer(self.next_arrival_time)
        
    @property
    def next_arrival_time(self):
        if self.simulator.time >= self._next_arrival_time:
            next_arrival_time = gen_exp_rv(self.ld) + self.simulator.time
            self._next_arrival_time = next_arrival_time
            
        return self._next_arrival_time

### Tracker
Handles cycle tracking, and pretty printing

In [267]:
class Tracker:
    def __init__(self):
        self.info = {}
        
    def track(self, queue_length, time):
        if queue_length not in self.info:
            self.info[queue_length] = []
            
        self.info[queue_length].append(time)
        
    def __str__(self):
        res = []
        for queue_length in sorted(self.info.keys()):
            vals = self.info[queue_length]
            
            num_cycles = len(vals) - 1
            
            if num_cycles == 0:
                continue
                
            span = vals[-1] - vals[0]
            mean_cycle_length = span / num_cycles
            
            res.append(
                f'Queue Length {queue_length}. {num_cycles} Cycles. Mean Cycle Length: {mean_cycle_length:.2f}'
            )
            
        return '\n'.join(res)

### Simulator
Orchestrator

In [268]:
class Simulator:
    def __init__(self, mu, ld, maxtime=9999999):
        self.customer_generator = CustomerGenerator(self, ld)
        self.stall = Stall(self, mu)
        self.maxtime = maxtime
        self.tracker = Tracker()
        self.time = 0
        
    def update_time(self, time):
        self.time = time
        
    def track(self):
        self.tracker.track(
            len(self.stall.queue),
            self.time
        )
    
    def iterate(self):
        def handle_arrival():
            customer = self.customer_generator.generate_customer()
            self.stall.add_customer(customer)
            
        def handle_departure():
            self.stall.serve_customer()
            
        next_arrival_time, next_departure_time = self.customer_generator.next_arrival_time, self.stall.next_departure_time
        is_arrival_first = next_arrival_time < next_departure_time
        
        if is_arrival_first:
            handle_arrival()
            self.update_time(next_arrival_time)
            
            logger.debug(f'Handled arrival at {self.time:.1f}')
        else:
            handle_departure()
            self.update_time(next_departure_time)
            
            logger.debug(f'Handled departure at {self.time:.1f}')
            
        self.track()
            
        
            
    def run(self):
        while self.time <= self.maxtime:
            self.iterate()
            


In [273]:
mu = 30 # seconds
ld = 40 # seconds
s = Simulator(mu, ld, 10000)
s.run()
print(s.tracker)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [179]:
logging.getLogger().setLevel(2)

Assuming the new customer's first wish is to get food from stall $s$

Let Q_i be the queue length of stall $i$

Let $c \in [0, 1]$, where $c$ is a parameter of the simulation determining the willingness of a random customer to queue.

\begin{align*}
\text{Queue joined} = \begin{cases} 
      \text{s} & \text{if }U(0, 1) \leq c^{Q_s}\\
      \min \limits_{Q_{s'} \in S}  s'  & \text{o.w.}\\
   \end{cases}
\end{align*}

INFO:root:lol
